In [1]:
import pandas as pd
import numpy as np

In [2]:
basepath = "/home/jonathan/tesis/1er_avance/ArchivosCSV/DataFiltradoFebrero2020.csv"
data = pd.read_csv(basepath, sep=',',dtype={"fecha": str,
                                        "origen": str,
                                        "destino": str},index_col='fecha',parse_dates=True, usecols=[0,1,2,7,8,9])          

data = data[data.distancia >= "002-005"]


basepath = "/home/jonathan/tesis/1er_avance/ArchivosCSV/DataFiltradoMarzo2020_1.csv"
data2 = pd.read_csv(basepath, sep=',',dtype={"fecha": str,
                                        "origen": str,
                                        "destino": str},index_col='fecha',parse_dates=True, usecols=[0,1,2,7,8,9])          

data2 = data2[data2.distancia >= "002-005"]
data2.head(10)

,origen,destino,periodo,distancia,viajes
fecha,,,,,
2020-03-01,19024,0200304,14,100+,7.774
2020-03-01,19024,0501901_AD,9,100+,7.130
2020-03-01,19024,08298,21,100+,7.456
2020-03-01,19024,0905909,13,100+,10.763
2020-03-01,19024,13087,8,100+,6.826
2020-03-01,19024,16027_AM,12,050-100,13.652
2020-03-01,19024,1808702,14,100+,4.954
2020-03-01,19024,1808707,17,100+,8.084
2020-03-01,19024,19024,0,002-005,5.626


In [3]:
data2.shape

(10245914, 5)

In [4]:
data.index.freq = data.index.inferred_freq
serieDia = data.resample('24H').sum()
serieDia.index

data2.index.freq = data2.index.inferred_freq
serieDia2 = data2.resample('24H').sum()
serieDia2.index

DatetimeIndex(['2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04',
               '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08',
               '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12',
               '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16'],
              dtype='datetime64[ns]', name='fecha', freq='24H')

In [5]:
data['trayecto'] = data['origen'] + '-' + data['destino']
data2['trayecto'] = data2['origen'] + '-' + data2['destino']
data2.head(5)

,origen,destino,periodo,distancia,viajes,trayecto
fecha,,,,,,
2020-03-01,19024,0200304,14,100+,7.774,19024-0200304
2020-03-01,19024,0501901_AD,9,100+,7.130,19024-0501901_AD
2020-03-01,19024,08298,21,100+,7.456,19024-08298
2020-03-01,19024,0905909,13,100+,10.763,19024-0905909
2020-03-01,19024,13087,8,100+,6.826,19024-13087


In [ ]:
df_Total = pd.DataFrame({})
reindex=data.reset_index()
contador = 0

for fecha in serieDia.index:   
    filtrar = reindex.loc[reindex.loc[:,'fecha'] == fecha]
    print(filtrar.shape)
    table_A = pd.pivot_table(filtrar, values=['viajes','origen','destino','fecha'], index=["periodo","trayecto"], aggfunc={'viajes':np.sum,'origen': 'first','destino': 'first','fecha': 'first'})
    reindex2 = table_A.reset_index()
    table_B = pd.pivot_table(reindex2, values=['viajes'], index=["periodo"], columns=["trayecto"], aggfunc={'viajes':'first'}, fill_value=0)
    print(table_A.shape)
    print(table_B.shape)
    
    if table_B.index.shape[0] != 24:
        j = 0
        for i in range(24):
            if table_B.index[i] != (i+j):
                print (reindex.fecha.iloc[0],i+j,"origen sin registros!")
                table_B.loc[i+j] = 0
                j += 1
            if (i + j) == 23: break
        table_B = table_B.sort_index() 
    
    table_B["ds"] = pd.date_range(reindex.fecha.iloc[0], "%s 23:00:00" % (reindex.fecha.iloc[0]), freq="H")  #fecha.iloc[0] devuelve "20200214"-"20200215"... y la hora en formato 24h      
    table_B = table_B.set_index("ds") #Fija como indice la columna ds       
    table_B = table_B.fillna(0)       #Datos sin registros los pondrá en 0
    
    df_Total = df_Total.append(table_B)
    df_Total = df_Total.fillna(0)
    
    contador = contador+1
    if contador == 2:
        break
    #break
print(df_Total.shape)

In [14]:
reindex=data.reset_index()
#prueb = pd.date_range(reindex.fecha.iloc[0], "%s 23:00:00" % (reindex.fecha.iloc[0]), freq="H")
prueb = pd.date_range(serieDia.index[0], "%s 23:00:00" % (serieDia.index[0]), freq="H")
print(prueb)

DatetimeIndex(['2020-02-14 00:00:00', '2020-02-14 01:00:00',
               '2020-02-14 02:00:00', '2020-02-14 03:00:00',
               '2020-02-14 04:00:00', '2020-02-14 05:00:00',
               '2020-02-14 06:00:00', '2020-02-14 07:00:00',
               '2020-02-14 08:00:00', '2020-02-14 09:00:00',
               '2020-02-14 10:00:00', '2020-02-14 11:00:00',
               '2020-02-14 12:00:00', '2020-02-14 13:00:00',
               '2020-02-14 14:00:00', '2020-02-14 15:00:00',
               '2020-02-14 16:00:00', '2020-02-14 17:00:00',
               '2020-02-14 18:00:00', '2020-02-14 19:00:00',
               '2020-02-14 20:00:00', '2020-02-14 21:00:00',
               '2020-02-14 22:00:00', '2020-02-14 23:00:00'],
              dtype='datetime64[ns]', freq='H')


In [12]:
print(reindex.fecha)

0          2020-02-14
1          2020-02-14
2          2020-02-14
3          2020-02-14
4          2020-02-14
              ...    
12000163   2020-02-29
12000164   2020-02-29
12000165   2020-02-29
12000166   2020-02-29
12000167   2020-02-29
Name: fecha, Length: 12000168, dtype: datetime64[ns]


In [ ]:
reindex=data2.reset_index()

for fecha in serieDia2.index:   
    filtrar = reindex.loc[reindex.loc[:,'fecha'] == fecha]
    print(filtrar.shape)
    table_A = pd.pivot_table(filtrar, values=['viajes','origen','destino','fecha'], index=["periodo","trayecto"], aggfunc={'viajes':np.sum,'origen': 'first','destino': 'first','fecha': 'first'})
    reindex2 = table_A.reset_index()
    table_B = pd.pivot_table(reindex2, values=['viajes'], index=["periodo"], columns=["trayecto"], aggfunc={'viajes':'first'}, fill_value=0)
    print(table_A.shape)
    print(table_B.shape)
    
    if table_B.index.shape[0] != 24:
        j = 0
        for i in range(24):
            if table_B.index[i] != (i+j):
                print (reindex.fecha.iloc[0],i+j,"origen sin registros!")
                table_B.loc[i+j] = 0
                j += 1
            if (i + j) == 23: break
        table_B = table_B.sort_index() 
    
    table_B["ds"] = pd.date_range(reindex.fecha.iloc[0], "%s 23:00:00" % (reindex.fecha.iloc[0]), freq="H")  #fecha.iloc[0] devuelve "20200214"-"20200215"... y la hora en formato 24h      
    table_B = table_B.set_index("ds") #Fija como indice la columna ds       
    table_B = table_B.fillna(0)       #Datos sin registros los pondrá en 0
    
    df_Total = df_Total.append(table_B)    
    df_Total = df_Total.fillna(0)
    #break
print(df_Total.shape)

In [ ]:
#df_febrero.iloc[72:92,59:57000]

In [ ]:
#hola = reindex2.loc[reindex2.loc[:,'trayecto'] == '19024-19046_AM']
#hola.head(25)

In [ ]:
#filtrar2 = filtrar.loc[filtrar.loc[:,'periodo'] == 0]
#filtrar2.iloc[0:24,:]

In [ ]:
df_Total.to_csv("Trayectos_Periodo_Referencia.csv")